In [1]:
from bs4 import BeautifulSoup
import urllib
import re

import urllib2
import json

In [2]:
#general format is https://www.kickstarter.com/projects/creatorname/campaignname

#Creating a function to grab detailed text information about the project from the project's kicstarter page
#Passing project id, url, and currency of funds requested from the meta data of successful project URLs
# URLS from './data/project_successful_meta.json' created by the project_metadata.ipyth program
def get_one_project_det(pid, url, currency, pcounter):
    #Creating beautiful soup object from URL
    try:
        
        r = urllib.urlopen(url)
        soup = BeautifulSoup(r)
        
        #Need to obtain the donation values and the reward/gifts associated with each donation value
        #Reading in tag with multiple pieces of info on this
        b=list(soup.find_all(class_="NS_projects__rewards_list js-project-rewards"))
        #Donation dollar amount group
        reward_value_list = [reward.string.split()[0] for reward in b[0]("h5",class_="mb1")]
        #Number of backers associated with Donation dollar amount group
        backer_list = [backers.string.split()[0]+' backers' for backers in b[0]("span", class_="num-backers mr1")]
        #Reward/gift associated with Donation dollar amount group
        reward_item_list = [item("p")[0].get_text() for item in b[0]("div",class_="desc h5 mb2 break-word")]
        
        #removing currency sign from reward value
        reg = re.compile("\d.*",flags=re.U)
        reward_value_list = [reg.findall(reward.string.split()[0])[0] for reward in b[0]("h5",class_="mb1")]
    
        #Text associated with the risks of the project--outlining project hurdles
        c=list(soup("div", class_="mb6 mb2"))
        risk=[item("p") for item in c][0]
        
       #Creating a dictionary for each project id 
        project_det = dict()
       #Project title 
        project_det["title"] = soup.find_all("meta", property="og:title")[0].get("content")
       #Blurb about project 
       # project_det["blurb1"] = soup.find_all(property=re.compile("og:description"))[0].get("content")
       #Creator's URL hompage
        project_det["creator_url"] = soup.find_all(property=re.compile("kickstarter:creator"))[0].get("content")
       #Total number of backers
        project_det["no_backers"] = list(soup("div", class_="NS_projects__spotlight_stats")[0])[1].string
       #Project goal amount---min value needed to be raised for successful project 
        project_det["goal"] = reg.findall(list(soup.find_all("span", class_="money " +currency.lower()+ " no-code")[2])[0].string)
       #Number of dollars raised during campaign 
        project_det["no_dollars_raised"] = list(soup("div", class_="NS_projects__spotlight_stats")[0])[3].string
       #Information about donation amount groups, number of backer for group and gift associated with group 
        project_det["reward_backer_tup"] = (zip(reward_value_list, backer_list,reward_item_list))
       #Creator's detailed information about risks involved with the project 
        project_det["risk"] = [para.get_text() for para in risk]
       #project id 
        project_det["id"] = pid
       #project url 
        project_det["url"] = url
        
        return(project_det)
    
    except:
        print pcounter
        return({pcounter: url})

In [4]:
#open url information
fd = open('./data/project_successful_meta.json', 'r')
text = fd.read()
fd.close()

pmeta = json.loads(text)

In [5]:
#call function created above to grab project details and loop through project for every url in pmeta file
# for homework purposes, limited this to 500 urls, but we'll pull more for final project
project_succ_det = [get_one_project_det(project_header["id"], project_header["url"],project_header["currency"], pcounter) for pcounter, project_header in enumerate(pmeta[0:2000])]

865
1019
1217
1324
1325
1326


In [20]:
project_succ_det[1386]

{'blurb1': u"A stunning new Acoustic Album from PANIC ROOM!!! We'll be capturing rare & intimate versions of some of the band's most beloved songs\u2026",
 'creator_url': 'https://www.kickstarter.com/profile/1276076804',
 'goal': [u'8,000'],
 'id': 549321147,
 'no_backers': u'385 backers',
 'no_dollars_raised': u'\xa315,793',
 'reward_backer_tup': [(u'5',
   u'16 backers',
   u'\u2726 PANIC ROOM - The Acoustic Album -\nDownload only'),
  (u'10', u'45 backers', u'\u2726 PANIC ROOM - The Acoustic Album'),
  (u'15', u'76 backers', u'\u2726 PANIC ROOM - The Acoustic Album - Signed!'),
  (u'35',
   u'174 backers',
   u'\u2726 PANIC ROOM - The Acoustic Album - DELUXE EDITION'),
  (u'50',
   u'6 backers',
   u'\u2726 PANIC ROOM - The Acoustic Album - DELUXE EDITION + STUDIO POSTERS'),
  (u'55', u'22 backers', u'\u2605 DELUXE EDITION - BRONZE PACKAGE'),
  (u'70', u'1 backers', u'\u2605 DELUXE EDITION - BRONZE PACKAGE \u271aPLUS!'),
  (u'75', u'12 backers', u'\u2605 DELUXE EDITION - SILVER PACK

In [9]:
pmeta[1]

{u'author': u'Jared Lerner',
 u'backers_count': 35,
 u'blurb': u'With their unique design and superior construction, the Just Live leggings fit great, look great, and do not cost an arm and a leg',
 u'category': {u'id': 263, u'slug': u'fashion/apparel'},
 u'currency': u'USD',
 u'deadline': 1429761879,
 u'goal': 15000.0,
 u'id': 119504178,
 u'launched_at': 1427169879,
 u'location': {u'country': u'US', u'name': u'Hallandale', u'state': u'FL'},
 u'name': u'Just Live- The First Premium Leggings at a Low Cost',
 u'pledged': 15245.0,
 u'spotlight': True,
 u'state': u'successful',
 u'url': u'https://www.kickstarter.com/projects/1428920854/just-live-the-first-premium-leggings-at-a-low-cost/description'}

In [10]:
project_succ_det[1]

{'blurb1': 'With their unique design and superior construction, the Just Live leggings fit great, look great, and do not cost an arm and a leg',
 'creator_url': 'https://www.kickstarter.com/profile/1428920854',
 'goal': [u'15,000'],
 'id': 119504178,
 'no_backers': u'35 backers',
 'no_dollars_raised': u'$15,245',
 'reward_backer_tup': [(u'5',
   u'5 backers',
   u'Thanks for supporting Just Live! We will send you Just Live branded stickers.  We will also list you among our early supporters on our website and send you an email thanking you for your support.'),
  (u'25',
   u'0 backers',
   u'The Just Live Performance T-Shirt- You will get one of our performance t-shirts, perfect for women, men and children! You will also receive Just Live branded stickers, be listed as a supporter on our website, and receive a thank you email from us!'),
  (u'55',
   u'6 backers',
   u'Just Live Performance Capris (Early Bird Special)- You will receive one pair of high-performance "Race Time Capris" in 

In [16]:

#Save data to file for further use
project_meta_json = json.dumps(obj = project_succ_det, indent = 4)
#File to look at for sample data
fd = open("./data/project_successful_detail_1000to3000.json", "w")
fd.write(project_meta_json)
fd.close()